In [1]:
import pickle as pkl
import numpy as np

## Loading data

In [2]:
input_dim = 300

In [3]:
with open('Train_'+str(input_dim)+'.pkl', 'rb') as tr:
    X_train_300 = pkl.load(tr)
tr.close()
X_train_300 = X_train_300.reshape(X_train_300.shape[0], X_train_300.shape[1], 1)
X_train_300.shape

(87844, 300, 1)

In [4]:
with open('Val_'+str(input_dim)+'.pkl', 'rb') as val:
    X_val_300 = pkl.load(val)
val.close()
X_val_300 = X_val_300.reshape(X_val_300.shape[0], X_val_300.shape[1], 1)
X_val_300.shape

(1000, 300, 1)

In [5]:
with open('Test_'+str(input_dim)+'.pkl', 'rb') as test:
    X_test_300 = pkl.load(test)
test.close()
X_test_300 = X_test_300.reshape(X_test_300.shape[0], X_test_300.shape[1], 1)
X_test_300.shape

(1000, 300, 1)

In [6]:
with open('Train_twenty.pkl', 'rb') as tr:
    X_train_20 = pkl.load(tr)
tr.close()

In [7]:
with open('Val_twenty.pkl', 'rb') as val:
    X_val_20 = pkl.load(val)
val.close()

In [8]:
with open('Test_twenty.pkl', 'rb') as test:
    X_test_20 = pkl.load(test)
test.close()

In [9]:
with open('Train_labels.txt', 'r') as l:
    train_labels = l.read().split('\n')
l.close()
len(train_labels)

87844

In [10]:
with open('Val_labels.txt', 'r') as v:
    val_labels = v.read().split('\n')
v.close()
len(val_labels)

1000

In [11]:
with open('Test_labels.txt', 'r') as t:
    test_labels = t.read().split('\n')
t.close()
len(test_labels)

1000

In [12]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
enc.fit(train_labels)

LabelEncoder()

In [13]:
y_train_labels = enc.transform(train_labels)
y_train_labels.shape

(87844,)

In [14]:
y_val_labels = enc.transform(val_labels)
y_val_labels.shape

(1000,)

In [15]:
y_test_labels = enc.transform(test_labels)
y_test_labels.shape

(1000,)

In [18]:
import keras
from keras import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, Input, Concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import to_categorical
from keras import metrics
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from keras.callbacks import EarlyStopping

In [17]:
y_train = to_categorical(y_train_labels)
y_train.shape

(87844, 10)

In [19]:
y_val = to_categorical(y_val_labels)
y_val.shape

(1000, 10)

In [20]:
y_test = to_categorical(y_test_labels)
y_test.shape

(1000, 10)

In [21]:
from sklearn.utils.class_weight import compute_class_weight

In [22]:
# Instantiate the label encoder
le = LabelEncoder()

# Fit the label encoder to our label series
le.fit(list(y_train_labels))

# Create integer based labels Series
y_integers = le.transform(list(y_train_labels))

In [23]:
class_weights = compute_class_weight('balanced', np.unique(y_train_labels), y_train_labels)
class_weights_dict = dict(zip(le.transform(list(le.classes_)), class_weights))

In [24]:
# Parameters
batch_size = 512
epochs = 100
num_classes = 10

## Keras has its own method for making "ensemble"

In [26]:
inputA = Input(shape=(input_dim,1))
inputB = Input(shape=(20,))

CNN для эмбеддингов используется, в противовес тому, что говорили на собрании, есть статьи с arxiv аж с 2014 года. Данные я разделил.

In [28]:
x = Conv1D(26, kernel_size=(10), padding='same')(inputA) 
x = LeakyReLU(alpha=0.1)(x)
x = MaxPooling1D(2, padding='same')(x)
x = Dropout(0.1)(x)
x = Flatten()(x)
x = Model(inputs = inputA, outputs = x)

In [29]:
y = Dense(20, activation="relu")(inputB)
y = Dense(10, activation="relu")(y)
y = Model(inputs=inputB, outputs = y)

In [30]:
combined = Concatenate()([x.output, y.output])

In [31]:
z = Dense(32, activation='relu')(combined)
z = Dense(num_classes, activation='softmax')(z)

In [32]:
model = Model(inputs =[x.input, y.input], outputs = z)

In [35]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=[metrics.categorical_accuracy])
print(model.summary())

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 300, 1)       0                                            
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 300, 26)      286         input_3[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_2 (LeakyReLU)       (None, 300, 26)      0           conv1d_2[0][0]                   
__________________________________________________________________________________________________
max_pooling1d_2 (MaxPooling1D)  (None, 150, 26)      0           leaky_re_lu_2[0][0]              
____________________________________________________________________________________________

In [34]:
earlystop = EarlyStopping(monitor='val_categorical_accuracy', min_delta = 0.0001, patience=5, verbose=1, mode='auto')
callbacks_list = [earlystop]

In [36]:
model.fit([X_train_300, X_train_20], y_train, batch_size=batch_size, epochs=epochs, callbacks=callbacks_list,
          validation_data=([X_val_300, X_val_20], y_val), class_weight = class_weights_dict, verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 87844 samples, validate on 1000 samples
Epoch 1/100
87844/87844 [==============================] - 25s 290us/step - loss: 1.7335 - categorical_accuracy: 0.5081 - val_loss: 1.2870 - val_categorical_accuracy: 0.5890
Epoch 2/100
87844/87844 [==============================] - 25s 285us/step - loss: 1.1264 - categorical_accuracy: 0.6948 - val_loss: 0.9807 - val_categorical_accuracy: 0.6890
Epoch 3/100
87844/87844 [==============================] - 25s 282us/step - loss: 0.9270 - categorical_accuracy: 0.7402 - val_loss: 0.8669 - val_categorical_accuracy: 0.7280
Epoch 4/100
87844/87844 [==============================] - 26s 293us/step - loss: 0.8484 - categorical_accuracy: 0.7551 - val_loss: 0.8034 - val_categorical_accuracy: 0.7490
Epoch 5/100
87844/87844 [==============================] - 25s 281us/step - loss: 0.7986 - categorical_accuracy: 0.7661 - val_loss: 0.7771 - val_categorical_accuracy: 0.7700
Epoch 6/100
87844/87844 [==============================] - 25s 280us/step - loss:

In [39]:
scores = model.evaluate([X_val_300, X_val_20], y_val, verbose=0)
print('Val loss:', scores[0])
print('Val accuracy:', scores[1])

Val loss: 0.7412080345153809
Val accuracy: 0.782


In [40]:
y_pred = model.predict([X_val_300, X_val_20])
y_pred_classes = [[1 if c == max(a) else 0 for c in a] for a in y_pred]
y_pred_labels = [a.index(1) for a in y_pred_classes]
confusion_matrix(y_pred_labels, y_val_labels)
#y_val_labels

array([[69,  4,  2,  5,  1,  0,  2,  4,  0,  1],
       [ 0, 71,  2,  0,  0,  2,  1,  0,  0,  4],
       [ 9,  7, 88,  2,  2,  1,  0,  5,  1,  1],
       [ 8,  4,  0, 86,  0,  1,  5,  4,  0,  0],
       [ 0,  0,  0,  2, 80,  1,  0,  1,  5,  6],
       [ 3,  0,  0,  0,  0, 82,  6,  0,  0,  4],
       [ 4,  4,  1,  0,  1,  2, 79,  2,  2,  2],
       [ 1,  4,  2,  1,  2,  1,  2, 78,  5,  2],
       [ 2,  4,  2,  4,  5,  3,  3,  2, 75,  6],
       [ 4,  2,  3,  0,  9,  7,  2,  4, 12, 74]], dtype=int64)

Далеко не лучший из полученных результатов, просто я подбирал разные параметры, и тот результат пока при тех же параметрах не воспроизвёлся, видимо, элемент случайности. Указанный в таблице на самом деле был получен с этими параметрами.

In [41]:
test_scores = model.evaluate([X_test_300, X_test_20], y_test, verbose=0)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

Test loss: 0.8139278514385223
Test accuracy: 0.747


In [42]:
y_pred = model.predict([X_test_300, X_test_20])
y_pred_classes = [[1 if c == max(a) else 0 for c in a] for a in y_pred]
y_pred_labels = [a.index(1) for a in y_pred_classes]
confusion_matrix(y_pred_labels, y_test_labels)

array([[78,  4,  0,  4,  5,  2,  5,  6,  1,  3],
       [ 1, 65,  1,  1,  0,  0,  0,  2,  4,  4],
       [ 3,  6, 83,  3,  2,  1,  3,  4,  2,  3],
       [ 4,  7,  4, 76,  0,  0,  2,  7,  0,  4],
       [ 1,  1,  1,  0, 78,  0,  0,  1,  9,  4],
       [ 3,  1,  1,  6,  1, 86,  2,  3,  1,  2],
       [ 4,  4,  4,  5,  0,  6, 80,  5,  0,  1],
       [ 3,  3,  2,  2,  0,  3,  3, 64,  5,  1],
       [ 1,  7,  1,  2,  8,  2,  3,  5, 64,  5],
       [ 2,  2,  3,  1,  6,  0,  2,  3, 14, 73]], dtype=int64)

In [43]:
print(classification_report(y_pred_labels, y_test_labels))

              precision    recall  f1-score   support

           0       0.78      0.72      0.75       108
           1       0.65      0.83      0.73        78
           2       0.83      0.75      0.79       110
           3       0.76      0.73      0.75       104
           4       0.78      0.82      0.80        95
           5       0.86      0.81      0.83       106
           6       0.80      0.73      0.77       109
           7       0.64      0.74      0.69        86
           8       0.64      0.65      0.65        98
           9       0.73      0.69      0.71       106

    accuracy                           0.75      1000
   macro avg       0.75      0.75      0.75      1000
weighted avg       0.75      0.75      0.75      1000



In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot cnn.png', show_shapes=True, show_layer_names=True)